In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
# Шаг 1: Создание Dataset
class CaliforniaHousingDataset(Dataset):
    def __init__(self, data, target):
        self.data = data
        self.target = target

    def __len__(self):
        return len(self.target)

    def __getitem__(self, idx):
        return torch.tensor(self.data[idx], dtype=torch.float32), torch.tensor(self.target[idx], dtype=torch.float32)

In [4]:
# Загрузка данных
data = fetch_california_housing()
X, y = data.data, data.target

In [5]:
# Нормализация данных
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [6]:
# Разделение данных на train и test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=13)

In [7]:
# Создание Dataset
train_dataset = CaliforniaHousingDataset(X_train, y_train)
test_dataset = CaliforniaHousingDataset(X_test, y_test)

In [8]:
# Шаг 2: Обертка в DataLoader
batch_size = 64
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [9]:
# Шаг 3 и 4: Архитектура сети с BatchNorm и Dropout
class CaliforniaHousingModel(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size, dropout_rate=0.2):
        super(CaliforniaHousingModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.bn1 = nn.BatchNorm1d(hidden_size1) # BatchNorm1d слой
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout_rate) # Dropout слой
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.bn2 = nn.BatchNorm1d(hidden_size2) # BatchNorm1d слой
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout_rate) # Dropout слой
        self.fc3 = nn.Linear(hidden_size2, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        x = self.dropout2(x)
        x = self.fc3(x)
        return x

In [10]:
# Шаг 6: Обучение и сравнение сходимости
def train_model(model, dataloader, criterion, optimizer, num_epochs=50):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, targets in dataloader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), targets)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
        epoch_loss = running_loss / len(dataloader.dataset)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

In [11]:
# Шаг 7: Обучение и оценка моделей
input_size = X_train.shape[1]
hidden_size1 = 64
hidden_size2 = 32
output_size = 1
dropout_rate = 0.2

In [12]:
# Adam
model_adam = CaliforniaHousingModel(input_size, hidden_size1, hidden_size2, output_size, dropout_rate)
criterion = nn.MSELoss()
optimizer_adam = optim.Adam(model_adam.parameters(), lr=0.001)
train_model(model_adam, train_dataloader, criterion, optimizer_adam)

Epoch [1/50], Loss: 1.7981
Epoch [2/50], Loss: 0.6503
Epoch [3/50], Loss: 0.5785
Epoch [4/50], Loss: 0.5363
Epoch [5/50], Loss: 0.5163
Epoch [6/50], Loss: 0.5010
Epoch [7/50], Loss: 0.5025
Epoch [8/50], Loss: 0.4874
Epoch [9/50], Loss: 0.4659
Epoch [10/50], Loss: 0.4650
Epoch [11/50], Loss: 0.4555
Epoch [12/50], Loss: 0.4424
Epoch [13/50], Loss: 0.4318
Epoch [14/50], Loss: 0.4267
Epoch [15/50], Loss: 0.4169
Epoch [16/50], Loss: 0.4173
Epoch [17/50], Loss: 0.4186
Epoch [18/50], Loss: 0.4196
Epoch [19/50], Loss: 0.4108
Epoch [20/50], Loss: 0.4131
Epoch [21/50], Loss: 0.4073
Epoch [22/50], Loss: 0.4068
Epoch [23/50], Loss: 0.4017
Epoch [24/50], Loss: 0.4045
Epoch [25/50], Loss: 0.3955
Epoch [26/50], Loss: 0.3980
Epoch [27/50], Loss: 0.3957
Epoch [28/50], Loss: 0.4013
Epoch [29/50], Loss: 0.3963
Epoch [30/50], Loss: 0.3909
Epoch [31/50], Loss: 0.3916
Epoch [32/50], Loss: 0.3916
Epoch [33/50], Loss: 0.3926
Epoch [34/50], Loss: 0.3846
Epoch [35/50], Loss: 0.3901
Epoch [36/50], Loss: 0.3854
E

In [13]:
# RMSProp
model_rmsprop = CaliforniaHousingModel(input_size, hidden_size1, hidden_size2, output_size, dropout_rate)
optimizer_rmsprop = optim.RMSprop(model_rmsprop.parameters(), lr=0.001)
train_model(model_rmsprop, train_dataloader, criterion, optimizer_rmsprop)

Epoch [1/50], Loss: 0.8423
Epoch [2/50], Loss: 0.5871
Epoch [3/50], Loss: 0.5454
Epoch [4/50], Loss: 0.5100
Epoch [5/50], Loss: 0.4931
Epoch [6/50], Loss: 0.4675
Epoch [7/50], Loss: 0.4558
Epoch [8/50], Loss: 0.4496
Epoch [9/50], Loss: 0.4321
Epoch [10/50], Loss: 0.4261
Epoch [11/50], Loss: 0.4272
Epoch [12/50], Loss: 0.4203
Epoch [13/50], Loss: 0.4064
Epoch [14/50], Loss: 0.4096
Epoch [15/50], Loss: 0.4056
Epoch [16/50], Loss: 0.4048
Epoch [17/50], Loss: 0.4002
Epoch [18/50], Loss: 0.3940
Epoch [19/50], Loss: 0.3925
Epoch [20/50], Loss: 0.3951
Epoch [21/50], Loss: 0.3907
Epoch [22/50], Loss: 0.3851
Epoch [23/50], Loss: 0.3904
Epoch [24/50], Loss: 0.3860
Epoch [25/50], Loss: 0.3843
Epoch [26/50], Loss: 0.3858
Epoch [27/50], Loss: 0.3847
Epoch [28/50], Loss: 0.3897
Epoch [29/50], Loss: 0.3894
Epoch [30/50], Loss: 0.3848
Epoch [31/50], Loss: 0.3766
Epoch [32/50], Loss: 0.3780
Epoch [33/50], Loss: 0.3841
Epoch [34/50], Loss: 0.3812
Epoch [35/50], Loss: 0.3776
Epoch [36/50], Loss: 0.3798
E

In [14]:
# SGD
model_sgd = CaliforniaHousingModel(input_size, hidden_size1, hidden_size2, output_size, dropout_rate)
optimizer_sgd = optim.SGD(model_sgd.parameters(), lr=0.001, momentum=0.9)
train_model(model_sgd, train_dataloader, criterion, optimizer_sgd)

Epoch [1/50], Loss: 0.8993
Epoch [2/50], Loss: 0.6015
Epoch [3/50], Loss: 0.5572
Epoch [4/50], Loss: 0.5278
Epoch [5/50], Loss: 0.5102
Epoch [6/50], Loss: 0.4911
Epoch [7/50], Loss: 0.4943
Epoch [8/50], Loss: 0.4686
Epoch [9/50], Loss: 0.4762
Epoch [10/50], Loss: 0.4593
Epoch [11/50], Loss: 0.4598
Epoch [12/50], Loss: 0.4522
Epoch [13/50], Loss: 0.4477
Epoch [14/50], Loss: 0.4424
Epoch [15/50], Loss: 0.4331
Epoch [16/50], Loss: 0.4374
Epoch [17/50], Loss: 0.4342
Epoch [18/50], Loss: 0.4271
Epoch [19/50], Loss: 0.4217
Epoch [20/50], Loss: 0.4239
Epoch [21/50], Loss: 0.4307
Epoch [22/50], Loss: 0.4292
Epoch [23/50], Loss: 0.4256
Epoch [24/50], Loss: 0.4211
Epoch [25/50], Loss: 0.4194
Epoch [26/50], Loss: 0.4199
Epoch [27/50], Loss: 0.4190
Epoch [28/50], Loss: 0.4197
Epoch [29/50], Loss: 0.4203
Epoch [30/50], Loss: 0.4160
Epoch [31/50], Loss: 0.4166
Epoch [32/50], Loss: 0.4148
Epoch [33/50], Loss: 0.4173
Epoch [34/50], Loss: 0.4114
Epoch [35/50], Loss: 0.4099
Epoch [36/50], Loss: 0.4109
E

In [15]:
# Оценка качества моделей на тестовых данных
def evaluate_model(model, dataloader):
    model.eval()
    total_loss = 0.0
    with torch.no_grad():
        for inputs, targets in dataloader:
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), targets)
            total_loss += loss.item() * inputs.size(0)
    mean_loss = total_loss / len(dataloader.dataset)
    return mean_loss

In [16]:
print("Evaluation on test data:")
loss_adam = evaluate_model(model_adam, test_dataloader)
loss_rmsprop = evaluate_model(model_rmsprop, test_dataloader)
loss_sgd = evaluate_model(model_sgd, test_dataloader)

print(f"Adam: {loss_adam:.4f}")
print(f"RMSProp: {loss_rmsprop:.4f}")
print(f"SGD: {loss_sgd:.4f}")

Evaluation on test data:
Adam: 0.3477
RMSProp: 0.5025
SGD: 0.6508


## Выводы:

На основании предоставленных данных о тренировке и тестировании моделей с использованием различных оптимизаторов (Adam, RMSProp, SGD), можно сделать следующие выводы:

1. Сходимость моделей:
   - Adam: Модель обучалась быстрее и достигла более низкой ошибки уже к 50-й эпохе. Значение функции потерь уменьшалось почти равномерно с каждой эпохой.
   - RMSProp: Модель также достигла низкой ошибки, но обучалась медленнее по сравнению с Adam. Значение функции потерь немного скачкообразно снижалось в начале, но потом стало уменьшаться более стабильно.
   - SGD: Модель также показала схожую сходимость, но процесс обучения был более нестабильным, и в начале значения функции потерь могли сильно скачать. Однако со временем процесс стабилизировался, и функция потерь снижалась.

2. Качество предсказаний на тестовых данных:
   - Модель, обученная с помощью Adam, показала лучший результат с наименьшей ошибкой (loss_adam = 0.3477) на тестовых данных. Это свидетельствует о хорошей обобщающей способности этой модели.
   - Модель, обученная с помощью RMSProp, также показала хорошие результаты с низкой ошибкой (loss_rmsprop = 0.5025) на тестовых данных, но она была немного выше, чем у модели Adam.
   - Модель, обученная с помощью SGD, показала наихудший результат с более высокой ошибкой (loss_sgd = 0.6508) на тестовых данных. Возможно, этот оптимизатор требует тщательной настройки гиперпараметров или более длительного обучения.

3. Вывод:
   - Из результатов можно сделать вывод, что Adam и RMSProp показывают хорошие результаты на этой задаче, но Adam обучается быстрее и дает более низкую ошибку на тестовых данных.
   - SGD также дает приемлемый результат, но требует более тщательной настройки и может потребовать более длительного обучения для достижения схожего качества с Adam и RMSProp.

Cтоит отметить, что результаты могут немного различаться в каждом запуске из-за случайной инициализации параметров моделей и случайности в данных при использовании SGD.